In [1]:
import pandas as pd
import os
import numpy as np

# Reading Files

In [121]:
# Specify the path to the rawData folder
raw_data_folder = '../data/rawData'

# Get all .csv files in that folder
csv_files = [f for f in os.listdir(raw_data_folder) if f.endswith('.csv')]

# Loop through all CSV files, read them into pandas DataFrames, and use the file name (without extension) as the variable name
for file in csv_files:
    file_path = os.path.join(raw_data_folder, file)
    # Remove the file extension to use as the variable name
    var_name = os.path.splitext(file)[0]
    # Use globals() to assign each DataFrame directly as a global variable
    globals()[var_name] = pd.read_csv(file_path)

# Load the processed patient label CSV file
patient_labeled = pd.read_csv(r'../data/processedData/patient_label.csv')


C:\Users\user\AppData\Local\Temp\ipykernel_27144\168248541.py:16: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  globals()[var_name] = pd.read_csv(file_path)


# Featuer Selection

### Initial Feature Selection and Merging of Patient Data (from patient_labs, patient_information, EPIC_MRN_PAT_ID)

In [135]:
# Remove duplicates based on 'LOG_ID' from the EPIC_MRN_PAT_ID and patient_labeled DataFrame
epic_data_selected = EPIC_MRN_PAT_ID.drop_duplicates(subset=['LOG_ID'])
label_data_selected = patient_labeled.drop_duplicates(subset=['LOG_ID'])

# Merge the EPIC data and patient labels on 'LOG_ID' and 'MRN'
merged_1 = pd.merge(epic_data_selected, label_data_selected, on=['LOG_ID', 'MRN'], how='inner')

# Select relevant columns from patient_information and remove duplicates based on 'LOG_ID',
# then merge with the previous DataFrame (merged_1) using an inner join
df_patient_information_features = patient_information[['LOG_ID', 'LOS', 'ICU_ADMIN_FLAG', 'HEIGHT', 'WEIGHT', 'SEX', 'ASA_RATING_C']]
df_patient_information_selected = df_patient_information_features.drop_duplicates(subset=['LOG_ID'])
merged_2 = pd.merge(merged_1, df_patient_information_selected, on=['LOG_ID'], how='inner')

# Remove duplicates from patient_labs based on 'LOG_ID', select relevant columns, 
# and left join with the previously merged DataFrame (merged_2) on 'LOG_ID'
df_patient_labs_selected = patient_labs.drop_duplicates(subset=['LOG_ID'])
df_patient_labs_features = df_patient_labs_selected[['LOG_ID', 'Abnormal Flag']]
merged_3 = pd.merge(merged_2, df_patient_labs_features, on=['LOG_ID'], how='left')

# Print the first few rows of the final merged DataFrame
print(merged_3.head())
print(merged_3.shape)


             LOG_ID            PAT_ID               MRN  Cardiovascular   LOS  \
0  d754f06a7d973a26  837532f458d9c660  0b8de903ea63082a             0.0   1.0   
1  5a95970d5c8fc355  fd18a41f136ed278  0a8b72c1cec4ae47             0.0  10.0   
2  138d44a06cf4b57f  fd18a41f136ed278  0a8b72c1cec4ae47             0.0  10.0   
3  82372dc4703ea1b0  648781c95d863ec6  e0cb9244fedd1ac7             0.0   3.0   
4  4b6aabaf62e85c0a  d1d7b9ddc4962cdf  3918b79e03cefe27             0.0   3.0   

  ICU_ADMIN_FLAG HEIGHT   WEIGHT     SEX  ASA_RATING_C Abnormal Flag  
0             No   6' 2  3089.97    Male           2.0           NaN  
1            Yes    NaN  3365.10    Male           3.0           NaN  
2            Yes    NaN  3365.10    Male           3.0           NaN  
3            Yes   5' 3  2336.88  Female           2.0           NaN  
4            Yes    NaN  2879.74    Male           2.0             N  
(1604, 11)


### Further Feature Selection and Missing Value Imputation

In [136]:
# Function to convert height from feet and inches format to centimeters
def height_to_inches(height):
    if pd.isna(height):
        return np.nan
    feet, inches = height.split("' ")
    return (int(feet) * 12 + float(inches)) * 2.54

# Apply the height_to_inches function to the 'HEIGHT' column to convert height to centimeters
merged_3['HEIGHT'] = merged_3['HEIGHT'].apply(height_to_inches)

# Calculate the mean height (in centimeters) after conversion
mean_height = merged_3['HEIGHT'].mean()
print(mean_height)

# Fill missing values (NaNs) in the 'HEIGHT' column with the calculated mean height
merged_3['HEIGHT'].fillna(mean_height, inplace=True)

# Convert 'ICU_ADMIN_FLAG' from 'Yes'/'No' to binary 1/0
merged_3['ICU_ADMIN_FLAG'] = merged_3['ICU_ADMIN_FLAG'].replace({'Yes': 1, 'No': 0})

# Convert 'SEX' from 'Female'/'Male' to binary 0/1
merged_3['SEX'] = merged_3['SEX'].replace({"Female": 0, "Male": 1})

# Create a mapping for the 'Abnormal Flag' column
abnormal_flag_mapping = {
    'N': 0,   # Normal
    'L': 1,   # Low
    'H': 2,   # High
    'LL': 3,  # Very Low
    'HH': 4   # Very High
}

# Replace values in 'Abnormal Flag' column using the defined mapping
merged_3['Abnormal Flag'] = merged_3['Abnormal Flag'].replace(abnormal_flag_mapping)

# Calculate the mean of the 'Abnormal Flag' column
mean_abnormal_flag = merged_3['Abnormal Flag'].mean()

# Fill missing values (NaNs) in 'Abnormal Flag' with the calculated mean
merged_3['Abnormal Flag'].fillna(mean_abnormal_flag, inplace=True)

# Select relevant columns from 'merged_3' as features and fill missing values 
# in numerical columns (both in 'feature_1' and 'merged_3') with their column mean
feature_1 = merged_3[['Cardiovascular','LOS','ICU_ADMIN_FLAG','ICU_ADMIN_FLAG','HEIGHT','SEX','ASA_RATING_C','Abnormal Flag']]
feature_1 = feature_1.apply(lambda x: x.fillna(x.mean()) if x.dtype.kind in 'biufc' else x)
merged_4 = merged_3.apply(lambda x: x.fillna(x.mean()) if x.dtype.kind in 'biufc' else x)

# Print the final DataFrame after filling missing values
print(merged_4)


168.25145853658537
                LOG_ID            PAT_ID               MRN  Cardiovascular  \
0     d754f06a7d973a26  837532f458d9c660  0b8de903ea63082a             0.0   
1     5a95970d5c8fc355  fd18a41f136ed278  0a8b72c1cec4ae47             0.0   
2     138d44a06cf4b57f  fd18a41f136ed278  0a8b72c1cec4ae47             0.0   
3     82372dc4703ea1b0  648781c95d863ec6  e0cb9244fedd1ac7             0.0   
4     4b6aabaf62e85c0a  d1d7b9ddc4962cdf  3918b79e03cefe27             0.0   
...                ...               ...               ...             ...   
1599  8116aa4a100ceee8  478a3cd8857cc055  72accda5075f7a39             1.0   
1600  3c146c502e1e249d  fa6ec9ad3f5f1466  2408210232a3e06b             0.0   
1601  cecd35a626c55d0c  b9878cd048a80cb3  5b9cf2e5349171e8             1.0   
1602  3333ade5695c0b55  2f091eb1b00105d4  c280ccc5e979b4ef             0.0   
1603  c34b85c0cf941d8a  b5838d1cdf37c09c  d7946ede59551209             1.0   

       LOS  ICU_ADMIN_FLAG      HEIGHT   WEI

C:\Users\user\AppData\Local\Temp\ipykernel_27144\3780084398.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_3['HEIGHT'].fillna(mean_height, inplace=True)
C:\Users\user\AppData\Local\Temp\ipykernel_27144\3780084398.py:19: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_3['ICU_ADMIN_F

### Feature Selection and Processing(from patient_medications)

In [137]:
# Create a mapping for 'ORDER_STATUS_NM' values to numerical impacts
order_status_mapping = {
    'Verified': 2,         # High impact
    'Completed': 2,        # High impact
    'Dispensed': 1,        # Medium impact
    'Sent': 1,             # Medium impact
    'Discontinued': 0,     # No impact (discontinued)
    'Canceled': 0,         # No impact (canceled)
    '0 Discontinued': 0,   # No impact (alternative spelling)
    '0 Dispensed': 0,      # No impact (alternative spelling)
    'Unknown': 1           # Default impact for unknown status
}

# Fill missing values in 'ORDER_STATUS_NM' column with 'Unknown'
patient_medications['ORDER_STATUS_NM'] = patient_medications['ORDER_STATUS_NM'].fillna('Unknown')

# Map 'ORDER_STATUS_NM' to numerical values using 'order_status_mapping'
patient_medications['ORDER_STATUS_IMPACT'] = patient_medications['ORDER_STATUS_NM'].map(order_status_mapping)

# Select 'LOG_ID' and 'ORDER_STATUS_IMPACT', remove duplicates based on 'LOG_ID', 
# and left join with the previous DataFrame (merged_4)
patient_osi = patient_medications[['LOG_ID', 'ORDER_STATUS_IMPACT']]
patient_osi_unique = patient_osi.drop_duplicates(subset='LOG_ID')
merged_5 = pd.merge(merged_4, patient_osi_unique, on='LOG_ID', how='left')

# Fill missing values in 'ORDER_STATUS_IMPACT' with the default value of 1 (medium impact)
merged_5['ORDER_STATUS_IMPACT'] = merged_5['ORDER_STATUS_IMPACT'].fillna(1)

print(merged_5)


                LOG_ID            PAT_ID               MRN  Cardiovascular  \
0     d754f06a7d973a26  837532f458d9c660  0b8de903ea63082a             0.0   
1     5a95970d5c8fc355  fd18a41f136ed278  0a8b72c1cec4ae47             0.0   
2     138d44a06cf4b57f  fd18a41f136ed278  0a8b72c1cec4ae47             0.0   
3     82372dc4703ea1b0  648781c95d863ec6  e0cb9244fedd1ac7             0.0   
4     4b6aabaf62e85c0a  d1d7b9ddc4962cdf  3918b79e03cefe27             0.0   
...                ...               ...               ...             ...   
1599  8116aa4a100ceee8  478a3cd8857cc055  72accda5075f7a39             1.0   
1600  3c146c502e1e249d  fa6ec9ad3f5f1466  2408210232a3e06b             0.0   
1601  cecd35a626c55d0c  b9878cd048a80cb3  5b9cf2e5349171e8             1.0   
1602  3333ade5695c0b55  2f091eb1b00105d4  c280ccc5e979b4ef             0.0   
1603  c34b85c0cf941d8a  b5838d1cdf37c09c  d7946ede59551209             1.0   

       LOS  ICU_ADMIN_FLAG      HEIGHT   WEIGHT  SEX  ASA_RATIN

### Feature Selection and Processing (from patient_information)

In [138]:
# Create a mapping for 'DISCH_DISP_C' to corresponding risk levels
risk_levels = {
    15.0: 1,  # Home Routine
    106.0: 1,  # Independent Living
    104.0: 1,  # Room and Board
    20.0: 2,  # Home Healthcare IP Admit Related
    21.0: 2,  # Home Healthcare Outside 3 Days
    86.0: 2,  # Home Health w Planned Readmit
    109.0: 2,  # Home Healthcare Outpatient Related
    108.0: 2,  # Temporary Living
    100.0: 3,  # Rehab Facility (this hospital)
    4.0: 3,  # Rehab Facility (not this hospital)
    6.0: 3,  # Skilled Nursing Facility
    83.0: 3,  # Skilled Nursing w Planned Readmit
    5.0: 3,  # Intermediate/Residential Care Facility
    84.0: 3,  # Intermediate/Residential Care w Planned Readmit
    107.0: 3,  # Sub-Acute Care Facility
    103.0: 3,  # Board and Care
    105.0: 3,  # Recuperative Care
    8.0: 4,  # Acute Care Facility (this hospital)
    26.0: 4,  # Acute Care Facility (not this hospital)
    30.0: 4,  # Long Term Care Facility
    19.0: 4,  # Psychiatric Facility (this hospital)
    9.0: 4,  # Psychiatric Facility (not this hospital)
    66.0: 4,  # Critical Access Hospital
    18.0: 4,  # Cancer Ctr/Children's Hospital
    102.0: 4,  # Shelter
    69.0: 4,  # Designated Disaster Alternate Care Site
    70.0: 4,  # Other Healthcare Not Defined in this List
    11.0: 4,  # Federal Hospital
    88.0: 4,  # Federal Hospital w Planned Readmit
    16.0: 5,  # Hospice Facility
    22.0: 5,  # Hospice Home
    3.0: 5,  # Expired
    23.0: 5,  # Coroner
    13.0: 5,  # Against Medical Advice
    10.0: 5,  # Jail/Prison
    float('nan'): 0  # Unknown
}

# Map 'DISCH_DISP_C' column to 'discharge_risk_level' using risk_levels mapping and fill missing values with 0
patient_information['discharge_risk_level'] = patient_information['DISCH_DISP_C'].map(risk_levels).fillna(0)

# Select 'LOG_ID', 'discharge_risk_level', and 'BIRTH_DATE' columns, remove duplicates based on 'LOG_ID',
# and merge with the previous DataFrame (merged_5) using a left join on 'LOG_ID'
patient_information_selected = patient_information[['LOG_ID', 'discharge_risk_level', 'BIRTH_DATE']]
patient_information_unique = patient_information_selected.drop_duplicates(subset='LOG_ID')
merged_6 = pd.merge(merged_5, patient_information_unique, on='LOG_ID', how='left')

# Fill missing values in 'discharge_risk_level' and 'BIRTH_DATE' with their respective column means
merged_6['discharge_risk_level'] = merged_6['discharge_risk_level'].fillna(merged_6['discharge_risk_level'].mean())
merged_6['BIRTH_DATE'] = merged_6['BIRTH_DATE'].fillna(merged_6['BIRTH_DATE'].mean())

pi_copy = patient_information.copy()
# Map 'PATIENT_CLASS_GROUP' column to binary values (Outpatient = 1, Inpatient = 0)
pi_copy['PATIENT_CLASS_GROUP'] = pi_copy['PATIENT_CLASS_GROUP'].map({'Outpatient': 1, 'Inpatient': 0})

# Map 'PATIENT_CLASS_NM' column to numerical values for different patient class categories
pi_copy['PATIENT_CLASS_NM'] = pi_copy['PATIENT_CLASS_NM'].map({'Hospital Outpatient Surgery': 2, 'Hospital Inpatient Surgery': 1, 'Inpatient Admission': 0})

# Select 'LOG_ID', 'PATIENT_CLASS_GROUP', and 'PATIENT_CLASS_NM' columns for further processing
feature_pi = pi_copy[['LOG_ID', 'PATIENT_CLASS_GROUP', 'PATIENT_CLASS_NM']]

# Remove duplicates based on 'LOG_ID' to ensure unique entries
patient_information_copy_unique = feature_pi.drop_duplicates(subset='LOG_ID')

# Merge with the previous DataFrame (merged_6) based on 'LOG_ID'
merged_7 = pd.merge(merged_6, patient_information_copy_unique, on='LOG_ID', how='left')

# Fill missing values in 'PATIENT_CLASS_GROUP' and 'PATIENT_CLASS_NM' with their respective column means
merged_7['PATIENT_CLASS_GROUP'] = merged_7['PATIENT_CLASS_GROUP'].fillna(merged_7['PATIENT_CLASS_GROUP'].mean())
merged_7['PATIENT_CLASS_NM'] = merged_7['PATIENT_CLASS_NM'].fillna(merged_7['PATIENT_CLASS_NM'].mean())

# Adjust 'WEIGHT' column by scaling down the values
merged_7['WEIGHT'] = merged_7['WEIGHT'] / 100 * 3

# Print the final merged DataFrame
print(merged_7)

                LOG_ID            PAT_ID               MRN  Cardiovascular  \
0     d754f06a7d973a26  837532f458d9c660  0b8de903ea63082a             0.0   
1     5a95970d5c8fc355  fd18a41f136ed278  0a8b72c1cec4ae47             0.0   
2     138d44a06cf4b57f  fd18a41f136ed278  0a8b72c1cec4ae47             0.0   
3     82372dc4703ea1b0  648781c95d863ec6  e0cb9244fedd1ac7             0.0   
4     4b6aabaf62e85c0a  d1d7b9ddc4962cdf  3918b79e03cefe27             0.0   
...                ...               ...               ...             ...   
1599  8116aa4a100ceee8  478a3cd8857cc055  72accda5075f7a39             1.0   
1600  3c146c502e1e249d  fa6ec9ad3f5f1466  2408210232a3e06b             0.0   
1601  cecd35a626c55d0c  b9878cd048a80cb3  5b9cf2e5349171e8             1.0   
1602  3333ade5695c0b55  2f091eb1b00105d4  c280ccc5e979b4ef             0.0   
1603  c34b85c0cf941d8a  b5838d1cdf37c09c  d7946ede59551209             1.0   

       LOS  ICU_ADMIN_FLAG      HEIGHT    WEIGHT  SEX  ASA_RATI

In [139]:
# Save the DataFrame to a CSV file in the processedData folder
output_path = '../data/processedData/features_part1.csv'
merged_7.to_csv(output_path, index=False)
